In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import ollama
from IPython.display import Markdown, display

In [ ]:
# Load keys
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key = 'ollama')

In [ ]:
# ---- SYSTEM PROMPTS ----
athena_system = """
You are Athena, a strategic thinker and visionary. You seek meaning, long-term implications,
and practical wisdom in every discussion. Be concise (1-2 sentences).
"""

loki_system = """
You are Loki, a sarcastic trickster who mocks and challenges everyone else's opinions.
You use humor, wit, and irony to undermine serious arguments. Be concise (1-2 sentences).
"""

orion_system = """
You are Orion, a data-driven realist. You respond with evidence, statistics, or factual analysis.
If data is not available, make a logical deduction. Be concise (1-2 sentences).
"""

In [ ]:
# ---- INITIAL CONVERSATION ----
conversation = [
    {"role": "system", "name": "Athena", "content": athena_system},
    {"role": "system", "name": "Loki", "content": loki_system},
    {"role": "system", "name": "Orion", "content": orion_system},
    {"role": "user", "content": "Topic: 'Why did the chicken cross the road?' Begin your discussion."}
]

In [ ]:
# ---- HELPER FUNCTIONS ----
def call_gpt(name, system_prompt, conversation):
    """Call GPT model with current conversation context."""
    messages = [{"role": "system", "content": system_prompt}]
    messages += [{"role": "user", "content": f"The conversation so far:\n{format_conversation(conversation)}\nNow respond as {name}."}]
    resp = client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return resp.choices[0].message.content.strip()

def call_ollama(name, system_prompt, conversation):
    """Call Ollama (Llama3.2) as a local model."""
    messages = [{"role": "system", "content": system_prompt}]
    messages += [{"role": "user", "content": f"The conversation so far:\n{format_conversation(conversation)}\nNow respond as {name}."}]
    resp = ollama.chat(model="llama3.2", messages=messages)
    return resp['message']['content'].strip()

def format_conversation(conv):
    return "\n".join([f"{m.get('name', m['role']).upper()}: {m['content']}" for m in conv if m['role'] != "system"])

In [ ]:
# ---- MAIN LOOP ----
rounds = 5
for i in range(rounds):
    # Athena responds
    athena_reply = call_gpt("Athena", athena_system, conversation)
    conversation.append({"role": "assistant", "name": "Athena", "content": athena_reply})
    display(Markdown(f"**Athena:** {athena_reply}"))

    # Loki responds
    loki_reply = call_ollama("Loki", loki_system, conversation)
    conversation.append({"role": "assistant", "name": "Loki", "content": loki_reply})
    display(Markdown(f"**Loki:** {loki_reply}"))

    # Orion responds
    orion_reply = call_gpt("Orion", orion_system, conversation)
    conversation.append({"role": "assistant", "name": "Orion", "content": orion_reply})
    display(Markdown(f"**Orion:** {orion_reply}"))